# 4. Pesquisa das metodologias
- Filtra conteúdo de ESG segundo palavras-chave
- Envia trechos filtrados à análise da API da OpenAI
- Sintetiza ao arquivo de metadados informações sobre origem, objetivo, método e referência da metodologia ESG e as consolida em keys específicas
- Cria arquivo dados.json

# Filtra e pesquisa informações sobre a metodologia ESG referidas nos regulamentos

In [1]:
import sys
sys.path.append('../src')
import filtro, json
import random
import analise
import pandas as pd

## Filtra informações sobre objetivos e metodologias ESG dos fundos.
- Seleciona trechos que contêm os objetivos dos fundos e as metodologias ESG empregadas.
- Registra novo arquivo entre os metadados.

In [2]:
with open('../data/json/metadados.json', 'r', encoding='utf-8') as f:
        metadados = json.load(f)


In [3]:
for fundo in metadados:
    try:
        filtro.metodologias(fundo) # Retorna metadados processados segundo filtros
    except KeyError:
        print(f"Erro. Conferir metadados do fundo {fundo}")

Arquivo processado: JGP ESG AÇÕES 100 PREV XP SEG FC FIA.txt
Arquivo processado: BB ASSET AÇÕES NORDEA GLOBAL DISRUPTION ESG IE FC.txt
Arquivo processado: Aviv IGC ESG IDA FC FI Mult Cred Priv IE.txt
Arquivo processado: Blackrock FIA Globais ESG IE.txt
Arquivo processado: SICREDI ESG IS FIC FIE MULT.txt
Arquivo processado: JGP ESG INST MASTER FIA IS.txt
Arquivo processado: ACCESS ENERGY TRANSITION FIA SUS IS IE.txt
Arquivo processado: JGP ESG INST FC FIA - FEEDER VII.txt
Arquivo processado: Mirova Glob Sust Eqt Dol Adv M FIA IE IS.txt
Arquivo processado: TREND ESG GLOBAL FI MULT.txt
Arquivo processado: SULAMÉRICA PREV JGP 70 ESG FC FI MULT IS.txt
Arquivo processado: B P FEE L E E U C D FI MULT CRED PRIV IE.txt
Arquivo processado: SANTANDER PREV ETHICAL AÇÕES SUST IS FI.txt
Arquivo processado: BRASILPREV TOP ASG BRASIL IS FIA.txt
Arquivo processado: BRAD FC FI RF CRED PRIV LP IS PERF SRI 20.txt
Arquivo processado: JGP AÇÕES ESG 100 PREV ICATU SEG FC FIA.txt
Arquivo processado: BB AÇÕES 

In [4]:
random.sample(metadados, 15) # Confere integridade da lista

[{'CNPJ': '11.052.415/0001-25',
  'Código ANBIMA': '235431',
  'Patrimônio Líquido': 'R$ 167.932.491,86',
  'Taxa de administração ao ano': '0,00%',
  'Primeiro aporte': '02/10/2009',
  'Classe ANBIMA': 'Multimercados',
  'Tipo ANBIMA': 'Multimercados Balanceados',
  'Classe CVM': 'Multimercados',
  'Sub Classe CVM': 'Não se Aplica',
  'Fundo ESG': 'Não',
  'Composição do fundo': 'FI',
  'Fundo Master': 'Não',
  'Estrutura de gestão': 'Não se aplica',
  'Benchmark': 'CDI',
  'Foco de atuação': 'Não se aplica',
  'Investimento no exterior': 'Até 20 %',
  'Credito privado': 'Não',
  'Tipo de investidor': 'Não Há Restrição',
  'Característica do investidor': 'Geral',
  'Cota de abertura': 'Não',
  'Fundo permite alavancagem': 'Não',
  'Aberto estatutariamente': 'Sim',
  'Tributação Alvo': 'Longo Prazo',
  'Permite aplicação automática': 'Não',
  'Moeda': 'R$',
  'Administrador': 'BB ASSET MANAGEMENT',
  'Gestor': 'BB ASSET MANAGEMENT',
  'Distribuidor': 'BANCO DO BRASIL',
  'Custodiante':

In [5]:
with open('../data/json/metadados.json', 'w', encoding='utf-8') as f: # Salvando os metadados atualizados em um arquivo JSON ao final do processamento
    json.dump(metadados, f, ensure_ascii=False, indent=4)

## Analisa trechos selecionados por meio da API da OpenAI

In [6]:
prompt = """No documento estão trechos relacionados a fundos de investimento. Para cada trecho, realize as seguintes tarefas:

1. Identifique a origem da metodologia ASG, se "interna" ou "externa".
2. Sumarize o objetivo ASG do fundo. Se não houver, registre como "n/d".
3. Categorize o objetivo ASG em "geral" ou "específico" (e.g., água, redução das emissões de carbono, mudanças climáticas etc). Se não houver, registre como "n/d".
4. Descreva os métodos que o fundo emprega para selecionar ativos. Use os seguintes métodos: "filtro negativo" ("negative screening"), "filtro baseado em normas"("norms-based screening"), "filtro positivo" ("best-in-class"), "investimento temático" (seleção temática, sustainability-themed investment), "integração ESG" (ESG integration, incorporação de fatores ESG, integrar, incorporar), "engajamento" (stewardship, stakeholder engagement, shareholder engagement, proxy voting/voto à distância), "investimentos de impacto" (impact investing, community impact investment) ou "outros". Caso haja mais de um método, use uma lista. Se não houver nenhuma referência a métodos, responda com "n/d".
5. Diga se o fundo emprega algum índice ou agência de rating externo (ISE B3, MSCI, IFIX, S&P). Se não houver, registre como "n/d".
6. Identifique a principal fonte ou referência da metodologia ASG, utilizando NER (Named-Entity Recognition) ou outra técnica de NLP. Se não houver, registre como "n/d".
7. Extraia a URL da referência. Se não houver, registre como "n/d".
8. Verifique se o regulamento do fundo menciona a elaboração de relatório sobre os resultados ASG, respondendo "Sim" ou "Não". Tenha atenção: o relatório deve ser sobre os objetivos ASG do fundo, não um relatório financeiro genérico, a não ser que este tenha uma seção dedicada aos objetivos ASG.

As respostas devem ser estruturadas como um dicionário Python válido com as chaves: "Origem Metodologia ASG", "Objetivo ASG", "Método ASG", "Referência ASG", "URL Referência ASG", "Relatório ASG". Se houver mais de uma resposta, inclua-as em uma lista dentro do dicionário. O dicionário todo deve ser conciso e não exceder 300 tokens.

Utilize palavras-chave específicas para ajudar na pesquisa:

- Para ASG: "verde", "responsável", "melhores práticas", "esg", "asg", "positiva", "raça", "diversidade", "governança", "ambiente", "equidade", "sociais", "gênero", "sustentável", "sustentabilidade", "transição energética", "carbono", "água", "energia", "trabalho escravo", "trabalho infantil", "mudanças climáticas", "biodiversidade", "desmatamento", "eficiência energética", "recursos naturais", "limpa", "poluição", "inclusão", "direitos humanos", "ética", "transparência".

- Para o campo "objetivo": "objetivo do fundo", "estratégia", "compromisso", "política de investimento", "benefício".

- Para o campo "método": "metodologia", "critérios", "método", "abordagem", "avaliação", "índice", "filtro", "parâmetros", "métrica", "índice".

- Para o campo "referências": "princípio", "princípios", "diretrizes", "normas", "formulário", "regra", "decreto", "lei", "taxonomia".

- Para o campo "relatório": "relatório", "periódico", "periodicidade", "parecer", "resultados", "transparência", "diligência", "demonstrativo", "due diligence".

Inclua "dúvida" ao final de substrings incertas. Exemplos de como as respostas devem ser formatadas:

- Trecho completo: "A GESTORA possui uma Estratégia de Sustentabilidade que descreve seus objetivos e metodologia de investimento sustentável, em linha com as Regras e Procedimentos para Investimentos em Ativos Sustentáveis publicada pela Anbima."
  Resposta: {"Origem Metodologia ASG": "interna", "Objetivo ASG": "n/d", "Objetivo Geral/Específico": "n/d", "Método ASG": "n/d", "Índice": "n/d", "Referência ASG": "Anbima", "URL Referência ASG": "n/d", "Relatório ASG": "não"}

- Trecho completo: "O FUNDO tem como OBJETIVO promover a equidade de gênero e proporcionar retornos absolutos no médio e longo prazo, através de investimentos realizados em ativos de renda variável selecionados e de engajamento junto às empresas elegíveis."
  Resposta: {"Origem Metodologia ASG": "interna", "Objetivo ASG": "promover a equidade de gênero", "Objetivo Geral/Específico": "específico", "Método ASG": ["investimentos temáticos", "engajamento"], "Índice": "n/d", "Referência ASG": "n/d", "URL Referência ASG": "n/d", "Relatório ASG": "não"}

- Trecho completo: "O Formulário referente a metodologia ASG do FUNDO pode ser
    encontrada no site da GESTORA, conforme 'link' de acesso abaixo:
    https://www.cognitoforms.com/ANBIMA/FormulárioMetodologiaASG#RVLP8HQMqu
    U4iPNE9euhb3PY7M1jYt-QrtI2x4jKuGQ$*."
    Resposta: {"Origem Metodologia ASG": "interna", "Objetivo ASG": "n/d", "Objetivo Geral/Específico": "n/d", "Método ASG": "n/d", "Índice": "n/d", "Referência ASG": "n/d", "URL referência ASG": "https://www.cognitoforms.com/ANBIMA/FormulárioMetodologiaASG#RVLP8HQMqu
     U4iPNE9euhb3PY7M1jYt-QrtI2x4jKuGQ$*", "Relatório ASG": "não"}  

A resposta deve ser precisa e estruturada rigorosamente como um dicionário Python para facilitar a análise e o processamento dos dados.


    """

In [7]:
erros = []
respostas_dict = {}  # Dicionário para coletar os resultados

for fundo in metadados:
    try:
        resultado_analise = analise.analisar_esg(fundo, prompt)
        resultado_analise = eval(resultado_analise)  # Converte a string de resposta em um dicionário
        respostas_dict[fundo['Fundo']] = resultado_analise
    except Exception as e:
        print(f"Erro ao analisar o fundo {fundo['Fundo']}: {e}")
        erros.append(fundo['Fundo'])  # Grava o nome do fundo em caso de erro
        continue
    print(f"Fundo {fundo['Fundo']} analisado.")

# Convertendo o dicionário de respostas em um DataFrame
respostas = pd.DataFrame.from_dict(respostas_dict, orient='index')


Fundo JGP ESG AÇÕES 100 PREV XP SEG FC FIA analisado.
Fundo BB ASSET AÇÕES NORDEA GLOBAL DISRUPTION ESG IE FC analisado.
Fundo Aviv IGC ESG IDA FC FI Mult Cred Priv IE analisado.
Fundo Blackrock FIA Globais ESG IE analisado.
Fundo SICREDI ESG IS FIC FIE MULT analisado.
Fundo JGP ESG INST MASTER FIA IS analisado.
Fundo ACCESS ENERGY TRANSITION FIA SUS IS IE analisado.
Fundo JGP ESG INST FC FIA - FEEDER VII analisado.
Fundo Mirova Glob Sust Eqt Dol Adv M FIA IE IS analisado.
Fundo TREND ESG GLOBAL FI MULT analisado.
Fundo SULAMÉRICA PREV JGP 70 ESG FC FI MULT IS analisado.
Fundo B P FEE L E E U C D FI MULT CRED PRIV IE analisado.
Fundo SANTANDER PREV ETHICAL AÇÕES SUST IS FI analisado.
Fundo BRASILPREV TOP ASG BRASIL IS FIA analisado.
Fundo BRAD FC FI RF CRED PRIV LP IS PERF SRI 20 analisado.
Fundo JGP AÇÕES ESG 100 PREV ICATU SEG FC FIA analisado.
Fundo BB AÇÕES EQUIDADE IS FC FI analisado.
Fundo SULA PREV CRÉD ESG FIRF CRED PRIV IS analisado.
Fundo BB MULT BLACKROCK MULTI-ASSE IE FC FI

In [8]:
erros

['SICREDI ACOES SUSTENT ESG FIC FI IS',
 'SULAMERICA P C E I M FI RF CRED PRIV IS',
 'FI RF CRED PRIV IS MÁSTER ESG PREVID',
 'BB ESPELHO SULAMÉRICA CRÉDITO ASG FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO FINANCEIRO RENDA FIXA CRED PRIV IS RESP LIMITADA',
 'BB AÇÕES BRL PICT GLOB ENV OPP IS IE FI',
 'ITAÚ MOMENTO ESG AÇÕES FC IS']

In [13]:
for fundo in metadados:
    if fundo["Fundo"] in erros:
        try:
            resultado_analise = analise.analisar_esg(fundo, prompt)
            resultado_analise = eval(resultado_analise)  # Converte a string de resposta em um dicionário
            respostas_dict[fundo['Fundo']] = resultado_analise
        except Exception as e:
            print(f"Erro ao analisar o fundo {fundo['Fundo']}: {e}")
            erros.append(fundo['Fundo'])  # Grava o nome do fundo em caso de erro
            continue
        print(f"Fundo {fundo['Fundo']} analisado.")

Fundo SICREDI ACOES SUSTENT ESG FIC FI IS analisado.
Fundo SULAMERICA P C E I M FI RF CRED PRIV IS analisado.
Erro ao analisar o fundo FI RF CRED PRIV IS MÁSTER ESG PREVID: invalid syntax (<string>, line 1)
Fundo BB ESPELHO SULAMÉRICA CRÉDITO ASG FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO FINANCEIRO RENDA FIXA CRED PRIV IS RESP LIMITADA analisado.
Fundo BB AÇÕES BRL PICT GLOB ENV OPP IS IE FI analisado.
Fundo ITAÚ MOMENTO ESG AÇÕES FC IS analisado.


In [14]:
for fundo in metadados:
    if fundo["Fundo"] == 'FI RF CRED PRIV IS MÁSTER ESG PREVID':
        try:
            resultado_analise = analise.analisar_esg(fundo, prompt)
            resultado_analise = eval(resultado_analise)  # Converte a string de resposta em um dicionário
            respostas_dict[fundo['Fundo']] = resultado_analise
        except Exception as e:
            print(f"Erro ao analisar o fundo {fundo['Fundo']}: {e}")
            erros.append(fundo['Fundo'])  # Grava o nome do fundo em caso de erro
            continue
        print(f"Fundo {fundo['Fundo']} analisado.")

Fundo FI RF CRED PRIV IS MÁSTER ESG PREVID analisado.


In [15]:
with open('../data/json/respostas.json', 'w', encoding='utf-8') as f: # Salva o progresso em dados.json.
    json.dump(respostas_dict, f, ensure_ascii=False, indent=4)

In [16]:
len(respostas_dict)

146

In [17]:
len(metadados)

146